In [25]:
import pymongo
from pprint import pprint
from datetime import datetime

In [26]:
def utc_to_dt(utc: float) -> datetime:
    """Convert a unix time to a python datetime."""
    return datetime.utcfromtimestamp(int(utc))

In [6]:
# define mongo connection
conn = pymongo.MongoClient('localhost', 27017)
db = conn["drug_pricing"]
coll = db["praw"]

In [52]:
q = coll.find({ "$text": {"$search": "lawrence november 2019"} })

In [1]:
pprint(q[0])

Pretty printing has been turned OFF


In [54]:
str(utc_to_dt(1271701857))

'2010-04-19 18:30:57'